In [18]:
import numpy as np
from iminuit import Minuit
from scipy.integrate import quad

import constants

### Inutile ici: on n'utilise pas Pantheon+ 

n_panth = 1701
cov_mat = np.array(constants.cov_mat).reshape(n_panth, n_panth)

def dL(z, omega, H0):
    borne = (omega/(1-omega)) ** (1/3)
    X = 1 / (H0 * np.sqrt(1-omega))
    def integrale(z_i):
        res, _ = quad(lambda t: 1/np.sqrt(1 + t**3), borne, borne * (1+z_i))
        return constants.C * (1+z_i) * X * res
    return np.array([integrale(z_i) for z_i in np.atleast_1d(z)])

def mu(z, omega, H0):
    return 5 * np.log10(dL(z, omega, H0)) + 25

def chi2(omega, H0, M):
    delta_mu = np.empty(n_panth)

    for i in range(n_panth):
        if constants.is_calibrator_panth[i] == 0:
            delta_mu[i] = mu(constants.z_data_panth[i],omega, H0) - (constants.m_b_corr_panth[i] - M)
        else:
            delta_mu[i] = (constants.m_b_corr_panth[i] - M) - constants.ceph_dist_panth[i]

    delta_mu = mu(constants.z_data_panth, omega, H0) - constants.mu_data_panth
    return np.dot(np.dot(cov_mat, delta_mu), delta_mu)